In [13]:
import numpy as np
import pandas as pd

import pyvo
from astroquery.gaia import Gaia

In [14]:
def sample(x, size):
    x = np.random.shuffle(x)
    return x[:size]

In [17]:
def get_ngf21():
    tap_service = pyvo.dal.TAPService("http://TAPVizieR.u-strasbg.fr/TAPVizieR/tap/")
    QUERY = f"""select top 10000 GaiaDR3 as source_id, Plx, e_Plx
            from \"J/MNRAS/508/3877/maincat\" as ngf
            join \"J/A+A/674/A33/gspc-wd\" as gspc 
            on ngf.GaiaEDR3 = gspc.GaiaDR3
            where ngf.e_TeffH is not NULL and ngf.e_loggH is not NULL and ngf.e_TeffHe is not NULL and ngf.e_loggHe is not NULL
            and RAND() < 0.01"""
    ngfwds = tap_service.search(QUERY).to_table().to_pandas()
    gaiaquery = f"""select source_id, r_med_geo
                from external.gaiaedr3_distance
                where source_id in {tuple(ngfwds.source_id)}"""
    gaiadists = Gaia.launch_job_async(gaiaquery).get_results().to_pandas()
    return pd.merge(ngfwds, gaiadists, on="source_id")

def get_plx_data():
    

    ngf21 = get_ngf21()
    data = np.array([ngf21.Plx, ngf21.e_Plx]).T
    return data

In [20]:
tap_service = pyvo.dal.TAPService("http://TAPVizieR.u-strasbg.fr/TAPVizieR/tap/")
QUERY = f"""select top 10000 GaiaDR3 as source_id, Plx, e_Plx
        from \"J/MNRAS/508/3877/maincat\" as ngf
        join \"J/A+A/674/A33/gspc-wd\" as gspc 
        on ngf.GaiaEDR3 = gspc.GaiaDR3
        where ngf.e_TeffH is not NULL and ngf.e_loggH is not NULL and ngf.e_TeffHe is not NULL and ngf.e_loggHe is not NULL
        and RAND() < 0.01"""
ngfwds = tap_service.search(QUERY).to_table().to_pandas()

In [25]:
gaiaquery = f"""select source_id, r_med_geo
            from external.gaiaedr3_distance
            where source_id in {tuple(ngfwds.source_id)}"""
gaiadists = Gaia.launch_job_async(gaiaquery).get_results().to_pandas()

INFO: Query finished. [astroquery.utils.tap.core]


In [11]:
gaiadists

array([[4.90667985e-27, 2.61949776e-28],
       [6.10563685e-26, 2.48751098e-28],
       [3.75777612e-25, 3.01012800e-28],
       ...,
       [2.09655917e-27, 2.53117242e-28],
       [3.10313353e-26, 3.16193001e-28],
       [7.14224623e-25, 3.35275570e-28]])

In [12]:
sampled_indices = np.array([
            np.random.choice(prob.shape[0], p=prob, size=100)
            for col_probs in prob.T
        ])

ValueError: 'p' must be 1-dimensional